In [1]:
%matplotlib inline
import numpy as np, pandas as pd, scipy, matplotlib.pyplot as plt, networkx as nx, spacy
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
# Import gatsby the lazy way
gatsby="In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since. \"Whenever you feel like criticizing any one,\" he told me, \"just remember that all the people in this world haven't had the advantages that you've had.\" He didn't say any more but we've always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that. In consequence I'm inclined to reserve all judgments, a habit that has opened up many curious natures to me and also made me the victim of not a few veteran bores. The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men. Most of the confidences were unsought--frequently I have feigned sleep, preoccupation, or a hostile levity when I realized by some unmistakable sign that an intimate revelation was quivering on the horizon--for the intimate revelations of young men or at least the terms in which they express them are usually plagiaristic and marred by obvious suppressions. Reserving judgments is a matter of infinite hope. I am still a little afraid of missing something if I forget that, as my father snobbishly suggested, and I snobbishly repeat a sense of the fundamental decencies is parcelled out unequally at birth. And, after boasting this way of my tolerance, I come to the admission that it has a limit. Conduct may be founded on the hard rock or the wet marshes but after a certain point I don't care what it's founded on. When I came back from the East last autumn I felt that I wanted the world to be in uniform and at a sort of moral attention forever; I wanted no more riotous excursions with privileged glimpses into the human heart. Only Gatsby, the man who gives his name to this book, was exempt from my reaction--Gatsby who represented everything for which I have an unaffected scorn. If personality is an unbroken series of successful gestures, then there was something gorgeous about him, some heightened sensitivity to the promises of life, as if he were related to one of those intricate machines that register earthquakes ten thousand miles away. This responsiveness had nothing to do with that flabby impressionability which is dignified under the name of the \"creative temperament\"--it was an extraordinary gift for hope, a romantic readiness such as I have never found in any other person and which it is not likely I shall ever find again. No--Gatsby turned out all right at the end; it is what preyed on Gatsby, what foul dust floated in the wake of his dreams that temporarily closed out my interest in the abortive sorrows and short-winded elations of men."

parser = spacy.load('en')

# Parse Gatsby
gatsby = parser(gatsby)

# Divide text into sentences and sort list of strings
sentences = []
for span in gatsby.sents:
    sent = ''.join(gatsby[i].string for i in range(span.start, span.end)).strip()
    sentences.append(sent)
    
# Create Tfidf matrix
counter = TfidfVectorizer(lowercase=False,
                         stop_words=None,
                         ngram_range=(1,1),
                         analyzer=u'word',
                         max_df=0.5,
                         min_df=1,
                         max_features=None,
                         vocabulary=None,
                         binary=False)

# Apply the vectorizer
data_counts = counter.fit_transform(sentences)

In [13]:
# Calculate similarity
similarity = data_counts * data_counts.T

# Identifying highest ranked sentence
nx_graph = nx.from_scipy_sparse_matrix(similarity)
ranks = nx.pagerank(nx_graph, alpha=.85, tol=.00000001)

ranked = sorted(((ranks[i],s) for i,s in enumerate(sentences)), reverse=True)
print(ranked[0])

(0.07159770141782323, 'This responsiveness had nothing to do with that flabby impressionability which is dignified under the name of the "creative temperament"--it was an extraordinary gift for hope, a romantic readiness such as I have never found in any other person and which it is not likely I shall ever find again.')


In [4]:
# Keyword summarization
# Remove stop words and punctuation, and get a unique list
gatsby_filt = [word for word in gatsby if word.is_stop==False and (word.pos_=='NOUN' or word.pos_=='ADJ')]
words = set(gatsby_filt)

# Create a grid indicating whether words are within 4 places of the target
adjacency = pd.DataFrame(columns=words, index=words, data=0)

# Iterate through each word and determine which unique words are neighbors with it.\
for i, word in enumerate(gatsby):
    # Check word's next 4 neighbors are in the word list
    if any([word == item for item in gatsby_filt]):
        # Stops at the end of the string, if less than 4 words
        end = max(0, len(gatsby)-len(gatsby)-(i+5))
        # Potential neighbors
        nextwords = gatsby[i+1:end]
        # Filter to only select those in the list
        inset = [x in gatsby_filt for X in nextwords]
        neighbors = [nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # add 1 to adjacency matrix for neighbors of the target
        if neighbors:
            adjacency.loc[word,neighbors] = adjacency.loc[word,neighbors]+1

print("Done!")

Done!


In [7]:
# Running TextRank
nx_words = nx.from_numpy_matrix(adjacency.as_matrix())
ranks = nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# ID 5 highest ranked words
ranked = sorted(((ranks[i],s) for i,s in enumerate(words)), reverse=True)
print(ranked[:5])

[(0.007692307692307675, men), (0.007692307692307675, elations), (0.007692307692307675, sorrows), (0.007692307692307675, abortive), (0.007692307692307675, interest)]


/Users/jkovach/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
